In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
from keras.callbacks import EarlyStopping
from keras import optimizers

Using TensorFlow backend.


In [2]:
#read mother data
test_f308_7 = pd.read_csv("C:/test/foundation_308_data/foundation_308_7_test.csv", index_col = 0)
train_f308_7 = pd.read_csv("C:/test/foundation_308_data/foundation_308_7_train.csv", index_col = 0)

train_x = train_f308_7.drop(["cancer_code","result"], axis=1)
train_y = train_f308_7.result
test_x = test_f308_7.drop(["cancer_code", "result"], axis=1)
test_y = test_f308_7.result

In [7]:
print(train_x.shape)
sample_f308_7,features_f308_7 = train_x.shape

(13123, 308)


In [12]:
# 1) parameter setting
early_stopping = EarlyStopping(patience=10)
adam = keras.optimizers.Adam(lr=0.001)
input_drop_out_m_1 = 0
drop_out_m_1 = 0.5
layers = [1500, 200, 200, 200, 50]

# 2) model build
input_m_1 = keras.layers.Input(shape=(308,))
m_1_m_dp = keras.layers.Dropout(input_drop_out_m_1)(input_m_1)
for i in layers:
    m_1_m = keras.layers.Dense(i,activation='relu')(m_1_m_dp)
    m_1_m_dp = keras.layers.Dropout(drop_out_m_1)(m_1_m)
m_1_m_final = m_1_m_dp
output_m_1 = keras.layers.Dense(1, activation="sigmoid")(m_1_m_final)
m_1_model = keras.models.Model(inputs=input_m_1,outputs=output_m_1)
m_1_model.compile(optimizer=adam, 
                loss='binary_crossentropy',
                metrics=['accuracy'])

m_1_model.fit(train_x, train_y, validation_split=0.1, callbacks=[early_stopping], epochs=100, batch_size=100)
loss_f308_7, accuracy_f308_7 = m_1_model.evaluate(test_x,test_y)

m_1_model.save("C:/test/model_for_analysis.h5")

ValueError: could not broadcast input array from shape (308,1) into shape (308)

In [23]:
model_all.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 308)               0         
_________________________________________________________________
dense_25 (Dense)             (None, 1500)              463500    
_________________________________________________________________
dropout_21 (Dropout)         (None, 1500)              0         
_________________________________________________________________
dense_26 (Dense)             (None, 200)               300200    
_________________________________________________________________
dropout_22 (Dropout)         (None, 200)               0         
_________________________________________________________________
dense_27 (Dense)             (None, 200)               40200     
_________________________________________________________________
dropout_23 (Dropout)         (None, 200)               0         
__________

In [24]:
model_all.fit(train_x, train_y, validation_split=0.1, callbacks=[early_stopping], epochs=100, batch_size=100)
loss_f308_7, accuracy_f308_7 = model_all.evaluate(test_x,test_y)

model_all.save("C:/test/model_for_analysis.h5")

ValueError: could not broadcast input array from shape (308,1) into shape (308)

In [11]:
train_y.shape

(13123,)

In [14]:
train_y

AttributeError: 'Series' object has no attribute 'flat'

In [ ]:
print(loss_f308_7, accuracy_f308_7)

In [71]:
import shap
import numpy as np

# select a set of background examples to take an expectation over
background = train_x.iloc[np.random.choice(train_x.shape[0], 100, replace=False)]

# explain predictions of the model on four images
e = shap.DeepExplainer(model_all, background)
# …or pass tensors directly
# e = shap.DeepExplainer((model.layers[0].input, model.layers[-1].output), background)
shap_values = e.shap_values(test_x.values)

# plot the feature attributions
#shap.image_plot(shap_values, -test_x.iloc[1:5].values)

In [75]:
shap_results = pd.DataFrame(data=shap_values[0],columns = list(test_x),index=test_f308_7.patient)

In [76]:
shap_results.to_csv("shap_results.csv")